CODTECH Internship - Task 1: Big Data Analysis with Dask

This notebook demonstrates processing a large-ish dataset with **Dask** to show scalability. It uses a synthetic dataset so you can run it anywhere.

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

# create a synthetic CSV (simulates a large dataset)
N = 2000000
df = pd.DataFrame({
    'user_id': np.random.randint(1, 100000, size=N),
    'timestamp': pd.date_range('2023-01-01', periods=N, freq='T'),
    'value': np.random.randn(N).round(3),
    'category': np.random.choice(['A','B','C','D'], size=N, p=[0.2,0.5,0.2,0.1])
})
csv_path = '/mnt/data/task1_synthetic_large.csv'
df.to_csv(csv_path, index=False)
print('Wrote', csv_path)

Load with Dask and do scalable aggregations

In [ ]:
ddf = dd.read_csv('/mnt/data/task1_synthetic_large.csv')
print(ddf.head())

# basic aggregations
agg = ddf.groupby('category').value.agg(['count','mean','std']).compute()
agg

Time-based resampling and insights

In [ ]:
ddf['timestamp'] = dd.to_datetime(ddf['timestamp'])
ddf2 = ddf.set_index('timestamp')
resampled = ddf2['value'].resample('D').mean().compute()
resampled.head()

Save results and short interpretation

In [ ]:
agg.to_csv('/mnt/data/task1_aggregations_by_category.csv')
resampled.to_csv('/mnt/data/task1_daily_mean.csv')
print('Saved results to /mnt/data')

insights = '''
1. Category B has the most records (highest count) — indicates majority of events.
2. Daily average value fluctuates; inspect days with outliers for anomalies.
3. Use distributed cluster for >10M rows; Dask can scale horizontally.
'''
print(insights)